In [1]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from sklearn.model_selection import train_test_split

In [2]:
train_dataset = image.ImageDataGenerator(
    rescale = 1/255,
    rotation_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [3]:
train_generator = train_dataset.flow_from_directory(
    'Dataset3/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 252 images belonging to 2 classes.


In [4]:
val_dataset = image.ImageDataGenerator(rescale = 1/255)

In [5]:
val_generator = val_dataset.flow_from_directory(
    'Dataset3/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 54 images belonging to 2 classes.


In [6]:
train_generator.class_indices, val_generator.class_indices

({'Covid-19': 0, 'Normal': 1}, {'Covid-19': 0, 'Normal': 1})

In [7]:
model = Sequential()

model.add(Conv2D(64, kernel_size = (3,3), input_shape = (224,224,3), activation = 'relu'))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 32)      18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 32)      9248      
_________________________________________________________________
max_pooling2d

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs = 10,
    validation_data = val_generator,
    validation_steps=2
)

## Confusion Matrix

In [11]:
model = load_model('./model-final.h5')

In [15]:
DATA_PATH = '/Users/gauravverma/Library/Containers/com.apple.STMExtension.Mail/Data/Documents/Coding_Blocks/MACHINE LEARNING/my-own-projects/Covid_19/Dataset3/'
covid_images = glob.glob(DATA_PATH + 'Val/Covid-19/*')

x_val = np.zeros((54, 224, 224, 3), dtype = 'float')
y_val = np.zeros((54,), dtype = 'float')

count = 0

for i, img_file in enumerate(covid_images):
    img = cv2.imread(img_file)
    img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
    x_val[i] = img
    y_val[i] = 0.0
    count += 1
    print(f'Doing {i}', end = '\r')

print('Done Half', count)
normal_images = glob.glob(DATA_PATH + 'Val/Normal/*')
n = len(normal_images)
print('total images :',n)
count = 0

for i, img_file in enumerate(normal_images):
    img = cv2.imread(img_file)
    img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
    x_val[i+27] = img
    y_val[i+27] = 1.0
    count+=1
    print(f'Doing {i}', end = '\r')

x_val = x_val/255
all_images = x_val
all_target = y_val
print('Done Full', count)

Done Half 27
total images : 27
Done Full 27


In [23]:
y_pred = model.predict_classes(x_val).reshape(-1,)

In [24]:
from sklearn.metrics import confusion_matrix

In [40]:
dc = {1:'Covid-19', 0:'Normal'}
Y_true = [dc[l] for l in y_val]
Y_pred = [dc[l] for l in y_pred]

conf_mat = confusion_matrix(Y_true, Y_pred, labels = ['Covid-19', 'Normal'])
tp = conf_mat[0][0]
fp = conf_mat[1][0]
tn = conf_mat[1][1]
fn = conf_mat[0][1]

print('Confusion-Matrix:-\n', conf_mat)
print()
print('True-Positives :', tp)
print('False-Positives :', fp)
print('True-Negatives :', tn)
print('False-Negatives :', fn)

Confusion-Matrix:-
 [[24  3]
 [ 0 27]]

True-Positives : 24
False-Positives : 0
True-Negatives : 27
False-Negatives : 3
